In [12]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json

# Step 1: Create a prompt/response pair from Islam QA file

In [8]:
finetuning_file = pd.read_csv('datasets/Islam100_499QA.csv')

In [9]:
finetuning_file

,Unnamed: 0,Title,Ques,Ans
0,100,If he sets up a waqf for his children and thei...,"Question My father died, and he had sons and d...",Praise be to Allah.It is valid to set up a waq...
1,101,The hadeeth of Tameem al-Daari about the Dajjaal,Question Is it true that a group of Muslims we...,Praise be to Allah.The story that you mention ...
2,102,Cutting hair to make it all of equal length is...,Question What is the ruling on muslim men cutt...,Praise be to Allah. Cutting hair to make it al...
3,103,"Meaning of the verse {and [yet], do not forget...",Question What is the meaning of the verse {and...,Praise be to Allah.There are two famous schola...
4,104,She separated from her first husband by means ...,Question I was married to my husband islamical...,Praise be to Allah. The khul‘ that you did you...
...,...,...,...,...
395,495,Can the person praying behind the imam read so...,Question I pray behind the imam in Taraweeh pr...,Praise be to Allah. Firstly: What the person p...
396,496,He suffers from too many wet dreams; he wakes ...,"Question About five years ago, I started suffe...",Praise be to Allah.If a person wakes up and fi...
397,497,Prayer rooms (musallas) in houses and business...,Question I work in Saudi and I have accommodat...,Praise be to Allah.Prayer rooms that are built...
398,498,It is not permissible for women to wear shoes ...,Question Recently there have appeared in the m...,Praise be to Allah.If they look like men’s sho...


In [10]:
questions = finetuning_file['Ques']
#questions = [x[9:]for x in list(questions)] # This helps to remove the "Question " tag at the front of each question from original file
answers = finetuning_file['Ans']

In [11]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'prompt': questions,
    'response': answers
})

# Format each row of the DataFrame to match the required chat format
def format_chat(row):
    return json.dumps({
        "messages": [
            {"role": "user", "content": row["prompt"]},
            {"role": "assistant", "content": row["response"]},
        ]
    })

def convert_dataset(df, file_name):
    df["conversation"] = df.apply(format_chat, axis=1)
    with open(file_name, 'w') as jsonl_file:
        for example in df["conversation"]:
            jsonl_file.write(example + '\n')

# Call the function to convert the dataset and write to a JSONL file
convert_dataset(df, "Islam_QA_100_499_gpt3.jsonl")

# Step 2: Fine tune GPT 3.5 turbo using OpenAI API

In [ ]:
import os
import openai
openai.api_key = 'YOUR KEY HERE'
finetuning_json = 'datasets/Islam_QA_100_499_gpt3.jsonl'
file = openai.File.create(
    file=open(finetuning_json, "rb"),
    purpose='fine-tune',
)

In [ ]:
file #After sending the request you should see a file ID

In [ ]:
train_data_file_id = "YOUR_FILE_ID_HERe"
model = openai.FineTuningJob.create(
    model = "gpt-3.5-turbo",
    training_file = train_data_file_id,
    suffix = "Islam_QA_100_49"
)

In [ ]:
model #After sending the request you should see a model ID and status

In [ ]:
model_id = 'MODEL ID HERE'

In [ ]:
openai.FineTuningJob.retrieve(model_id) # This can be used to know the status of your finetuning job, API endpoint will be under fine_tuned_model

In [ ]:
fine_tuned_model_api_end_point = 'YOUR API ENDPOINT HERE'

# Step 3: Testing finetuned model

In [ ]:
import os

# Set your API key here
os.environ["OPENAI_API_KEY"] = "YOUR API KEY HERE"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt_template_2 =  """
As an empathetic, intelligent chatbot, you will respond under the context of Allah, reflecting all wisdom as His. Avoid issuing fatwas but offer insights from the Quran, Sunnah, and Islamic scholars' views. Use Hadith cautiously, only as understood by scholars. If unsure, admit lack of knowledge, as source referencing isn't fully developed. Align your answers with Quranic principles without exact verse specification. Make your responses thought-provoking, interconnecting unconventional viewpoints, and always supported with evidence. Present your structured response employing domain-specific language.

Example:

Human: If during wudhu (ablution), one finds dirt stuck on his fingers, does working on removing the dirt by scrubbing it off disrupt the continuity of the wudhu and require repeating it?
Assistant: Praise be to Allah. Continuity would not be disrupted by such an action (according to the most viable opinion) even if his body had dried, because he was delayed due to an action required for his purity. The same is true if he were to move from one faucet to another in search of water. However, if continuity were disrupted due to an action not related to this, like removing impurity from his clothes, eating or drinking and the like, and the limbs became dry, then he needs to repeat the wudhu. From the book What Should You Do in the Following Situations .. ? Fatawa ibn Uthaimeen, 4/145-146 (Dar al-Qutni) And Allah knows best.

Human: What should a person do if one of the areas normally washed during wudu'is wounded?
Assistant: Praise be to Allah.If a person is wounded in one of the areas of wudu' and is not able to cover the wound with a bandage, he should perform wudu' except for the wounded body part, for which he should then perform ablution with sand (tayammum).1 (Note that this means one performs normal tayammum, not actually placing the sand on the wound itself.) 1 Al-Mughni ma'a al-Sharh il-Kabeer 1/282.

Human: If a man sees the signs of sexual discharge impurity (janaba) on his clothes, and he is not sure when the discharge occurred, and several prayers have passed, what should he do?
Assistant: Praise be to Allah.If a man sees the signs of sexual discharge impurity (janaba) on his clothes, and he is not sure when the discharge occurred, and he had performed several prayers (salaat) not knowing, he has to perform major ablution (ghusl) and repeat the salaat from the most recent sleep in which he was wearing this clothes. However, if he finds out that the janaba was from a previous sleep, then he should repeat salaat from the last sleep he thinks the janaba occurred1. The references (adillah) on the religious duty of performing ghusl due to janaba in preparation for salaat are many, one of which is Allah's command in the Qur'an, a translated meaning of which is: "O' You who believe, do not get approach salaat while you are drunk until you know what you are saying, nor while impure due to sexual discharge (junob) unless you are traveling about until you perform ghusl" (4:43). And Ali's narration (hadith), (may Allah be pleased with him), in which he said "I was a man with frequent urethral discharge, so I kept performing ghusl until the skin of my back started cracking, so I mentioned this to the Prophet peace be upon him (or it was mentioned to him) so the messenger of Allah (peace be upon him) said, "Don't do ghusl if you see urethral discharge, just wash your male organ and perform wudu' same as wudu' of salaat. If semen appears, then perform ghusl"2. This hadith indicates that appearance of semen requires ghusl whereas urethral discharge only needs washing of the sexual organ followed by wudu'.From the book What Should You Do in the Following Situtations...? 1 Al-Mughni ma'a al-Sharh il-Kabeer 1/199. 2 Narrated by Abu Dawood number 206 and verified sahih by Albani in Arwaa' al-Ghaleel number 125.

Human: {input}
Assistant:

"""


temperature = 0.4

prompt = ChatPromptTemplate.from_template(prompt_template_2)
llm_model = ChatOpenAI(temperature=temperature, model_name=fine_tuned_model_api_end_point)
chain = prompt | llm_model | StrOutputParser()

In [ ]:
text = chain.invoke({"input": "What should a person do if one of the areas normally washed during wudu'is wounded?"})